# Part 5: Verifier


### Load the Previous Session
Run the next cell to load the variables we'll use in this part. Since it repeats everything done in previous parts - it will take a while to run.

In [1]:
import time
from tutorial_sessions import part1, part3 

def decommit_on_fri_layers(idx, channel):
    for layer, merkle in zip(fri_layers[:-1], fri_merkles[:-1]):
        length = len(layer)
        idx = idx % length
        sib_idx = (idx + length // 2) % length        
        channel.send(str(layer[idx]))
        channel.send(str(merkle.get_authentication_path(idx)))
        channel.send(str(layer[sib_idx]))
        channel.send(str(merkle.get_authentication_path(sib_idx)))
    channel.send(str(fri_layers[-1][0]))

def decommit_on_query(idx, channel): 
    assert idx + 16 < len(f_eval), f'query index: {idx} is out of range. Length of layer: {len(f_eval)}.'
    channel.send(str(f_eval[idx])) # f(x).
    channel.send(str(f_merkle.get_authentication_path(idx))) # auth path for f(x).
    channel.send(str(f_eval[idx + 8])) # f(gx).
    channel.send(str(f_merkle.get_authentication_path(idx + 8))) # auth path for f(gx).
    channel.send(str(f_eval[idx + 16])) # f(g^2x).
    channel.send(str(f_merkle.get_authentication_path(idx + 16))) # auth path for f(g^2x).
    decommit_on_fri_layers(idx, channel)    

def decommit_fri(channel):
    for query in range(3):
        # Get a random index from the verifier and send the corresponding decommitment.
        decommit_on_query(channel.receive_random_int(0, 8192-16-1), channel)

start = time.time()
start_all = start
print("Generating the trace...")
_, _, _, _, _, _, _, f_eval, f_merkle, _ = part1()
print(f'{time.time() - start}s')
start = time.time()
print("Generating the composition polynomial and the FRI layers...")
fri_polys, fri_domains, fri_layers, fri_merkles, channel = part3()
print(f'{time.time() - start}s')
start = time.time()
print("Generating queries and decommitments...")
decommit_fri(channel)
print(f'{time.time() - start}s')
start = time.time()
# print(channel.proof)
print(f'Overall time: {time.time() - start_all}s')
print(f'Uncompressed proof length in characters: {len(str(channel.proof))}')

Generating the trace...
13.96262812614441s
Generating the composition polynomial and the FRI layers...
20.6886248588562s
Generating queries and decommitments...
0.7515149116516113s
Overall time: 35.40322494506836s
Uncompressed proof length in characters: 46153


# Verify
verify the random numbers in the channel.
TODO: reduce proof size by removing the random numbers from the proof and recomputing them as in here

In [2]:
from channel import Channel

verifier_channel = Channel()
for i, c in enumerate(channel.proof):
    if c.startswith('send:'):
        verifier_channel.send(c[len('send:'):])
#         print(f'{i} send')
    elif c.startswith('receive_random_field_element:'):
        verifier_channel.receive_random_field_element()
        assert verifier_channel.proof[-1] == c,f"r={r} c={c}"
#         print(f'{i} receive_random_field_element')
    elif c.startswith('receive_random_int:'):
        verifier_channel.receive_random_int(0, 8192-16-1)
        assert verifier_channel.proof[-1] == c,f"r={r} c={c}"
#         print(f'{i} receive_random_int')
    else:
        raise f"unknown channel prefix {c}"
        
print('Success!')

Success!


utilites to read information from the channel's proof

In [3]:
channel_idx = 0

def get_s(s='send:'):
    global channel_idx
    assert channel_idx < len(channel.proof)
    assert channel.proof[channel_idx].startswith(s)
    v = channel.proof[channel_idx][len(s):]
    channel_idx += 1
    return v

def get_f():
    v = get_s('receive_random_field_element:')
    return FieldElement(int(v))

def get_i():
    v = get_s('receive_random_int:')
    return int(v)

### channel information from part 1

In [4]:
mt_root = get_s()  # of the 8K points on the trace poly

### channel information from part 2

In [5]:
from field import FieldElement

alpha0 = get_f()
alpha1 = get_f()
alpha2 = get_f()

In [6]:
cp_mt_root = get_s()  # of the 8K points on the composite poly

### channel information from part 3

Below the function `cp` returns the value of the composite polynomial at `x`. Note that this is the only place in the verifier where we enter the constrains (algorithm, parameters and result) of what we want to prove
* `x` is value on the evaluation domain of the polynomials (8K points)
* `f=f0` is the value of the interpolated polynimal `p` of the trace at the point `x`
* `f1` is the value of the  interpolated polynimal at `g * x` this point has an index which is +8 the index of `x`
* `f2` is the value of the  interpolated polynimal at `g * g * x` this point has an index which is +16 the index of `x`

In [7]:
g = FieldElement.generator() ** (3 * 2 ** 20)

def p0(x,f):
    return (f-1)/(x-1)

def p1(x,f):
    return (f-2338775057)/(x-g**1022)

def p2(x,f0,f1,f2):
    return (f2 - f1*f1 - f0*f0)/((1 - x**1024) / ((x - g**1021) * (x - g**1022) * (x - g**1023)))

def cp(x,f0,f1,f2):
    return alpha0*p0(x,f0) + alpha1*p1(x,f0) + alpha2*p2(x,f0,f1,f2)

In [8]:
alphas = []
fri_merkles_root = [cp_mt_root]
while channel.proof[channel_idx].startswith('receive_random_field_element:') and channel.proof[channel_idx+1].startswith('send:'):
    alphas.append(get_f())
    fri_merkles_root.append(get_s())
fri_constant = FieldElement(int(get_s()))

### channel information from part 4
Repeat query 3 times. In each query:
* read and merkle validate value of intepolated polynomal at a random point `x` (idx) and `g*x` (idx+8) and `g*g*x` (idx+16)
* compute the value of the composite polynoimal at `x`
* read and merkle validate the value of the first fri layer at `x`.
* compare the fri value at `x` to the computed value.  Note that we are not doing the same for `-x`
* compute next layer value
* repeat on next layer at same index which is `x**2`

In [9]:
#channel_idx = 26  # reset for repeated debug runs
from merkle import verify_decommitment

idxs = []
w = FieldElement.generator()
h = w ** ((2 ** 30 * 3) // 8192)

def read_verify(idx,root):
    v = get_s()
    # TODO using eval on a string coming from an untrusted source is dangerous
    assert verify_decommitment(idx,v,eval(get_s()),root)
    return FieldElement(int(v))

for query in range(3):
#     print(query)
    idx = get_i()  # pick a random location in the domain
    assert idx < 8192
    
    # sample the interpolated poly of the trace `f` at 3 locations that are one distance apart in the trace domain
    # one step in the trace domain is 8 in the evaluation domain
    f0 = read_verify(idx,mt_root) # f(x)
    f1 = read_verify(idx+8,mt_root) # f(gx)
    f2 = read_verify(idx+16,mt_root) # f(g^2x)
    
    x0 = w * (h ** idx)
    next_cp0 = cp(x0,f0,f1,f2)  # compute the value of the composite poly `cp` from `f`
    length = 8192
    # TODO we dont need to send fri_merkles_root[-1] in the channel
    for i,(fri_merkle_root,alpha) in enumerate(zip(fri_merkles_root[:-1],alphas)):
#         print(f'i={i}')
        idx = idx % length
        x0 = w * (h ** idx)  # convert the index idx to a point in the evaluation domain. needed for cp_odd below
        x0 = x0 ** (2**i)
        cp0 = read_verify(idx,fri_merkle_root)
        assert cp0 == next_cp0

        sib_idx = (idx + length // 2) % length        
#         x1 = w * (h ** sib_idx)  # convert the index sib_idx to a point in the evaluation domain
#         x1 = x1 ** (2**i)
#         assert x0*x0 == x1*x1,'x0 != -x1'
        cp1 = read_verify(sib_idx,fri_merkle_root)
        # note we dont check cp1 because we dont have a computed value

        cp_even = (cp0 + cp1) / 2
        cp_odd = (cp0 - cp1) / 2
        cp_odd /= x0
        next_cp0 = cp_even + alpha * cp_odd

        length = length // 2
        
    assert fri_constant == next_cp0
    # TODO the proof implementation above gives us fri_constant again after each iteration. We dont need this
    assert fri_constant == FieldElement(int(get_s()))
    
assert channel_idx == len(channel.proof), 'we did not reach the end of the channel'

print('Success!')

Success!
